<a href="https://colab.research.google.com/github/1MperaDOR0/Gerador-de-Tamplates-de-WebSites/blob/main/Gerador_de_Tamplates_para_WebSites_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalando o SDK do Google
!pip install -q -U google-generativeai
!pip install gdown

# Importando bibliotecas necessárias de Python
import google.generativeai as genai
import numpy as np
import pandas as pd
import os
import gdown

# Fazendo a linkagem da API do Google AI Studio
GOOGLE_API_KEY = "COLE_SUA_API"  # Substitua pela sua API Key
genai.configure(api_key=GOOGLE_API_KEY)

# Definindo as configurações de geração de conteúdo
generation_config = {
    'candidate_count': 1,
    'temperature': 0.5,
}
# Definindo as configurações de segurança
safety_settings = {
    'Harassment': 'BLOCK_NONE',
    'Hate': 'BLOCK_NONE',
    'Sexual': 'BLOCK_NONE',
    'Dangerous': 'BLOCK_NONE',
}
# Inicializando o modelo
model = genai.GenerativeModel(
    model_name='gemini-1.0-pro',
    generation_config=generation_config,
    safety_settings=safety_settings
)

# Criando o histórico do chat
chat = model.start_chat(history=[])

# Melhorando a visualização
# Código disponível em https://ai.google.dev/tutorials/python_quickstart#import_packages
import textwrap
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Função para ler o conteúdo de um arquivo do Google Drive
def ler_arquivo(id_arquivo):
  url = f'https://drive.google.com/uc?id={id_arquivo}&export=download'
  output = 'temp_file'
  gdown.download(url, output, quiet=True)
  with open(output, 'r') as arquivo:
    conteudo = arquivo.read()
  return conteudo

# Carregando os documentos e gerando embeddings
documents = []
for i in range(1, 4):  # Loop para os documentos 1 a 3
  html_id = f'1OvAWXVIrbCx4iyGorLneMR9wYUj_vc4x'  # ID do arquivo HTML
  css_id = f'1vlfe0CpnialN4vx7NTP1mitBso7ncMFD'  # ID do arquivo CSS

  documento = {
      'Arquivo HTML': ler_arquivo(html_id),
      'Arquivo CSS': ler_arquivo(css_id),
      'embedding': None
  }
  documents.append(documento)

df = pd.DataFrame(documents)

# Gerando embeddings para cada documento
embedding_model = "models/embedding-001"
for index, row in df.iterrows():
  conteudo = row['Arquivo HTML'] + row['Arquivo CSS']  # Combinando HTML e CSS
  embedding = genai.embed_content(model=embedding_model, content=conteudo, task_type='RETRIEVAL_QUERY')['embedding']
  df.at[index, 'embedding'] = embedding

# Função para buscar o documento mais relevante
def gerar_e_buscar_consulta(prompt, df, embedding_model):
  embedding_da_consulta = genai.embed_content(model=embedding_model, content=prompt, task_type='RETRIEVAL_QUERY')['embedding']
  produtos_escalares = np.dot(np.stack(df['embedding'].values), embedding_da_consulta)
  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]

# Loop principal do chatbot
prompt = input('\nDigite seu prompt:\n')
while prompt.strip().lower() != "finalizar":
  documento_relevante = gerar_e_buscar_consulta(prompt, df, embedding_model)
  conteudo_html = documento_relevante['Arquivo HTML']
  conteudo_css = documento_relevante['Arquivo CSS']

  # Incluindo o contexto na mensagem:
  mensagem_completa = f"""
  {prompt}

  ## Contexto HTML:
  {conteudo_html}

  ## Contexto CSS:
  {conteudo_css}
  """

  response = chat.send_message(mensagem_completa)
  print('\nTemplate gerado:\n', response.text, '\n')
  prompt = input('Digite seu prompt: ')


Digite seu prompt:
Kethely

Template gerado:
 **Análise do Código HTML e CSS**

**HTML**

* O código HTML define a estrutura da página da web.
* Ele inclui elementos como `header`, `main` e `footer` para organizar o conteúdo.
* A seção `main` contém duas seções: "Notícias em Destaque" e "Outras Notícias".
* Cada seção de notícias contém uma lista de artigos de notícias, incluindo título, imagem e descrição resumida.

**CSS**

* O código CSS estiliza a aparência da página da web.
* Ele define estilos para elementos como cabeçalho, navegação, seções de notícias e rodapé.
* O CSS usa uma combinação de estilos embutidos e folhas de estilo externas (referenciadas no cabeçalho).
* O CSS usa técnicas como flexbox e transições para criar um layout responsivo e efeitos visuais.

**Estilos Específicos**

* **Notícias em Destaque:** A notícia principal é exibida em um bloco maior com um plano de fundo cinza. As notícias adicionais são exibidas em uma lista vertical.
* **Outras Notícias:** As not

KeyboardInterrupt: Interrupted by user